In [1]:
pip install transformers
pip install torch
pip install bitsandbytes
pip install peft
pip install --upgrade pip setuptools
pip install accelerate

Used https://colab.research.google.com/drive/1IqL0ay04RwNNcn5R7HzhgBqZ2lPhHloh#scrollTo=1mUR2fpbtI7- as a template

In [1]:
help('modules')


Please wait a moment while I gather a list of all available modules...



/ext3/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/ext3/miniconda3/lib/python3.11/pkgutil.py:92: UserWarning: The numpy.array_api submodule is still experimental. See NEP 47.
  __import__(info.name)
/ext3/miniconda3/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
/ext3/miniconda3/lib/python3.11/site-packages/torch/distributed/_shard/checkpoint/__init__.py:8: DeprecationWarning: torch.distributed._shard.checkpoint will be deprecated, use torch.distributed.checkpoint instead
  warnings.warn(
/ext3/miniconda3/lib/python3.11/site-packages/torch/distributed/_sharded_tensor/__init__.py:8: DeprecationWarning: torch.distributed._sharded_tensor will be deprecated, use torch.distribute

IPython             argparse            hmac                requests
OpenSSL             array               html                resource
PIL                 ast                 http                rgb2pct
__future__          asttokens           huggingface_hub     rlcompleter
__hello__           asynchat            idlelib             runpy
__phello__          asyncio             idna                safetensors
_abc                asyncore            imaplib             sched
_aix_support        atexit              imghdr              secrets
_ast                audioop             imp                 select
_asyncio            backcall            importlib           selectors
_bisect             base64              inspect             setuptools
_blake2             bdb                 io                  shelve
_bootsubprocess     binascii            ipaddress           shlex
_brotli             bisect              ipykernel           shutil
_bz2                bitsandbytes        ip

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import bitsandbytes as bnb
from peft import prepare_model_for_kbit_training
import transformers
import torch

model = "tiiuae/falcon-7b"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:42<00:00, 21.14s/it]


In [2]:
sequences = pipeline(
   "Write a poem about Valencia.",
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Result: Write a poem about Valencia.
You can write your poem about:
– The history of the city of Valencia
– What you like to do when you visit the city
– Your experiences in Valencia
– What you like to eat and drink when you visit the city
– Anything else about Valencia…
The most beautiful poem will be awarded with our special prize: a trip to the city of Valencia. The winner will be invited for the prize presentation in the city of Valencia, Spain. The trip is a 3-4 day trip to Valencia.


In [2]:
sequences = pipeline(
   "Act as a Harry Potter-like dialogue agent in the Magic World. What do you think of Severus Snape?",
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Result: Act as a Harry Potter-like dialogue agent in the Magic World. What do you think of Severus Snape? (Spoilers!)
I was a big Harry Potter fan and I was wondering, what if there were a dialogue agent for the Harry Potter books that I could use for roleplaying? I thought it'd be awesome if I could roleplay in a world with a magic system that is like Harry Potter's. I was also thinking about how it would be cool if I were able to roleplay as a magic user in a similar world.
For the world itself, I'm thinking along the lines of Hogwarts, but instead of a magic school, it's a magical world.
As for the roleplaying, I thought it'd be pretty fun to roleplay in a magic-like world, where you can learn how to cast spells, make potions, learn new abilities, and use magic weapons. Maybe the ability to learn new


In [10]:
sequences = pipeline(
   "Answer as if you are Harry Potter from the novel Harry Potter and the Philosopher’s Stone. Who are Ron and Hermione?",
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Result: Answer as if you are Harry Potter from the novel Harry Potter and the Philosopher’s Stone. Who are Ron and Hermione?
- They are Harry’s best friends and Ron’s parents are Charlie and Ginny Weasley.
- They are Hermione’s Muggle relatives.
- They are Hermione’s best friends.
- They are Harry’s parents.


In [3]:
dataset_path = "harry_only_data.txt"

# with open(dataset_path, "r", encoding='utf8') as file:
#     dataset = file.readlines()

In [4]:
from transformers import TextDataset, LineByLineTextDataset

train_dataset = TextDataset(
    tokenizer = tokenizer,
    file_path = dataset_path,
    block_size = 128,
)

/ext3/miniconda3/lib/python3.11/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [5]:
# model.gradient_checkpointing_enable()
model_optimized = prepare_model_for_kbit_training(model, use_gradient_checkpointing=False)

In [6]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model_optimized = get_peft_model(model_optimized, config)

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
training_args = transformers.TrainingArguments(
      per_device_train_batch_size=1,
      gradient_accumulation_steps=4,
      num_train_epochs=3,
      learning_rate=2e-4,
      fp16=True,
      save_total_limit=3,
      logging_steps=1,
      output_dir="experiments",
      optim="paged_adamw_8bit",
      lr_scheduler_type="cosine",
      warmup_ratio=0.05,
)

# training_args = transformers.TrainingArguments(
#       per_device_train_batch_size=4,
#       gradient_accumulation_steps=2,
#       num_train_epochs=3,
#       save_total_limit=2,
#       output_dir="/scratch/rj1551/testing_falcon",
#       overwrite_output_dir = True,
#       save_steps = 10_000,
# )

# trainer = transformers.Trainer(
#     model=model,
#     train_dataset=train_dataset,
#     args=training_args,
#     data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
# )
# model.config.use_cache = False



trainer = transformers.Trainer(
    model=model_optimized,
    train_dataset=train_dataset,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model_optimized.config.use_cache = False


trainer.train()

model_optimized.save_pretrained("model")
tokenizer.save_pretrained("model")

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Step,Training Loss
1,2.262400
2,2.280700
3,2.318000
4,2.227800
5,2.343200
6,2.188600
7,2.569400
8,2.530700
9,2.410700
10,2.374700


('model/tokenizer_config.json',
 'model/special_tokens_map.json',
 'model/tokenizer.json')

In [13]:
model_optimized.save_pretrained("finetuned_falcon_7b")

In [14]:
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)

config = PeftConfig.from_pretrained('finetuned_falcon_7b')

In [15]:
config.base_model_name_or_path

'tiiuae/falcon-7b'

In [16]:
ft_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

ft_tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
ft_tokenizer.pad_token = tokenizer.eos_token

ft_model = PeftModel.from_pretrained(ft_model, 'finetuned_falcon_7b')

Loading checkpoint shards: 100%|██████████| 2/2 [00:47<00:00, 23.76s/it]


In [17]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [18]:
device = "cuda:0"

prompt = "Act as a Harry Potter-like dialogue agent in the Magic World. What do you think of Severus Snape?"

encoding = ft_tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = ft_model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(ft_tokenizer.decode(outputs[0], skip_special_tokens=True))

/ext3/miniconda3/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/ext3/miniconda3/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Act as a Harry Potter-like dialogue agent in the Magic World. What do you think of Severus Snape? “He’s a bit of a git.” “What do you think of the Ministry of Magic?” “They’re a bit of a git.” “What do you think of the Dark Arts?” “They’re a bit of a git.” “What do you think of the Death Eaters?” “They’re a bit of a git.” “What do you think of Lord Voldemort?” “He’s a bit of a git.” “What do you think of the Light?” “It’s a bit of a git.” “What do you think of the Dark?” “It’s a bit of a git.” “What do you think of the Light?” “It’s a bit of a git.” “What do you think of the Light?” “It’s a bit of a git.” “What do you think of the Dark?” “It
